# Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Muchamad Aldi Firmansyah
- **Email:** aldif0317@gmail.com
- **ID Dicoding:** aldi_03

## Menentukan Pertanyaan Bisnis

- Bagaimana performa penjualan dan revenue perusahaan dalam beberapa bulan terakhir?
- Produk apa yang paling banyak dan paling sedikit terjual?
- Bagaimana demografi pelanggan yang kita miliki?

## Import Semua Packages/Library yang Digunakan

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set style untuk visualisasi
sns.set(style="whitegrid")

In [7]:
import os
print(os.getcwd())

C:\Users\Aldi\dicoding proyek\ecommerce_dataset\submission


In [8]:
os.listdir()

['.ipynb_checkpoints', 'dashboard', 'data', 'Proyek_Analisis_Data.ipynb']

## Data Wrangling

### Gathering Data

In [ ]:
# Path untuk menyimpan file di folder dashboard
output_path = "C:/Users/Aldi/dicoding proyek/ecommerce_dataset/submission/dashboard"

# Pastikan folder dashboard sudah ada
import os
os.makedirs(output_path, exist_ok=True)

# Simpan dataset ke dalam file CSV
main_df.to_csv(os.path.join(output_path, "main_data.csv"), index=False)

print("main_data.csv berhasil dibuat di folder dashboard!")

In [ ]:
# Import library yang dibutuhkan
import pandas as pd
import os

# Path folder dataset
dataset_path = "C:/Users/Aldi/dicoding proyek/ecommerce_dataset/submission/data"

# Load semua dataset
customers_df = pd.read_csv(os.path.join(dataset_path, "customers_dataset.csv"))
geolocation_df = pd.read_csv(os.path.join(dataset_path, "geolocation_dataset.csv"))
order_items_df = pd.read_csv(os.path.join(dataset_path, "order_items_dataset.csv"))
order_payments_df = pd.read_csv(os.path.join(dataset_path, "order_payments_dataset.csv"))
order_reviews_df = pd.read_csv(os.path.join(dataset_path, "order_reviews_dataset.csv"))
orders_df = pd.read_csv(os.path.join(dataset_path, "orders_dataset.csv"))
products_df = pd.read_csv(os.path.join(dataset_path, "products_dataset.csv"))
product_translation_df = pd.read_csv(os.path.join(dataset_path, "product_category_name_translation.csv"))
sellers_df = pd.read_csv(os.path.join(dataset_path, "sellers_dataset.csv"))

# Tampilkan beberapa baris pertama dari setiap dataset
datasets = {
    "Customers": customers_df,
    "Geolocation": geolocation_df,
    "Order Items": order_items_df,
    "Order Payments": order_payments_df,
    "Order Reviews": order_reviews_df,
    "Orders": orders_df,
    "Products": products_df,
    "Product Translation": product_translation_df,
    "Sellers": sellers_df
}

for name, df in datasets.items():
    print(f"Dataset: {name}")
    display(df.head())  # Gunakan display() jika di Jupyter Notebook, kalau di script pakai print(df.head())
    print("\n" + "="*50 + "\n")

**Insight:**
- Dataset berhasil dimuat ke dalam notebook.
- Setiap tabel memiliki peran berbeda dalam analisis data.r_items, dan order_payments memiliki keterkaitan untuk analisis penjualan.

### Assessing Data

In [ ]:
# List semua dataset dalam dictionary untuk mempermudah iterasi
datasets = {
    "customers": customers_df,
    "geolocation": geolocation_df,
    "order_items": order_items_df,
    "order_payments": order_payments_df,
    "order_reviews": order_reviews_df,
    "orders": orders_df,
    "products": products_df,
    "product_translation": product_translation_df,
    "sellers": sellers_df
}

# Cek informasi umum dari setiap dataset
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df.info())
    print("\n" + "="*50 + "\n")


**Insight:**
- Dataset dengan banyak missing values perlu diperiksa lebih lanjut.
- Pastikan tipe data sesuai (misal, kolom tanggal harus dalam format datetime).

In [ ]:
# Cek missing values
for name, df in datasets.items():
    print(f"Missing Values in {name}:")
    print(df.isnull().sum())
    print("\n" + "="*50 + "\n")

**Insight:**
- Jika banyak missing values, kita harus menentukan apakah perlu dihapus atau diisi.
- Kolom kategori atau tanggal yang kosong mungkin perlu diisi dengan nilai default.



In [ ]:
# Cek jumlah duplikasi
for name, df in datasets.items():
    duplicate_count = df.duplicated().sum()
    print(f"Dataset: {name} - Duplicate Rows: {duplicate_count}")

**Insight:**
- Jika ada baris duplikat, kita bisa menghapusnya menggunakan df.drop_duplicates().
- Dataset seperti orders dan customers seharusnya tidak memiliki duplikasi karena memiliki ID unik.

In [ ]:
# Cek nilai unik dalam kolom order_status
print("Unique order_status values:", orders_df["order_status"].unique())

# Cek apakah ada produk tanpa kategori
print("Jumlah produk tanpa kategori:", products_df["product_category_name"].isnull().sum())

# Cek apakah ada tanggal yang tidak masuk akal
invalid_dates = orders_df[orders_df["order_delivered_customer_date"] < orders_df["order_purchase_timestamp"]]
print("Jumlah order dengan tanggal tidak valid:", len(invalid_dates))

📌 Insight:
- Jika ada status yang aneh (misal, "unknown"), kita bisa mengelompokkannya ke dalam kategori lain.
- Jika ada produk tanpa kategori, kita bisa menggantinya dengan "unknown" atau mencari kategorinya berdasarkan produk lain.
- Jika ada tanggal order tidak valid, kita harus mengecek dan memperbaiki kesalahan input.

### Cleaning Data

#### Menghapus Kolom yang Tidak Diperlukan

In [ ]:
# Hapus kolom yang tidak relevan
geolocation_df.drop(columns=["geolocation_zip_code_prefix"], inplace=True)

# Hapus kolom pada datasets lain jika tidak digunakan dalam analisis
order_reviews_df.drop(columns=["review_comment_title", "review_comment_message"], inplace=True)

# Cek hasil
geolocation_df.head()

📌 Insight:
- Menghapus kolom yang tidak akan digunakan mempercepat pemrosesan data.

#### Mengisi missing values yang bisa diperbaiki

In [ ]:
# Hapus baris dengan missing values di kolom penting (jika datanya sedikit)
orders_df.dropna(subset=["order_delivered_customer_date"], inplace=True)

# Isi missing values di kolom kategori produk dengan "unknown"
products_df["product_category_name"] = products_df["product_category_name"].fillna("unknown")

# Cek apakah masih ada missing values
print(products_df.isnull().sum())


📌 Insight:
- Order yang belum dikirim kemungkinan dibatalkan, jadi dihapus.
- Produk tanpa kategori diberi label "unknown" untuk tetap digunakan.

#### Menghapus data duplikat

In [ ]:
# Hapus data duplikat
for name, df in datasets.items():
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    after = df.shape[0]
    print(f"{name}: {before - after} duplikasi dihapus.")

📌 Insight:
- Data duplikat bisa menyebabkan perhitungan yang salah dalam analisis.

#### Memperbaiki inkonsistensi data

In [ ]:
# Filter order status hanya yang valid
valid_status = ["delivered", "shipped", "processing", "approved"]
orders_df = orders_df.loc[orders_df["order_status"].isin(valid_status)].copy()  # Tambahkan .copy()

# Konversi kolom tanggal ke format datetime
date_columns = ["order_purchase_timestamp", "order_delivered_customer_date"]
for col in date_columns:
    orders_df.loc[:, col] = pd.to_datetime(orders_df[col])

# Cek hasil
print(orders_df.dtypes)

📌 Insight:
- Menghapus status aneh menghindari kesalahan dalam analisis order.
- Format datetime memastikan kita bisa melakukan analisis waktu dengan benar.
- Kategori produk dalam lowercase menghindari perbedaan karena huruf besar/kecil.

## Exploratory Data Analysis (EDA)

### Explore Revenue & Penjualan

In [ ]:
# Tambahkan kolom bulan dan tahun
orders_df["order_purchase_month"] = orders_df["order_purchase_timestamp"].dt.to_period("M")

# Gabungkan order dengan item untuk mendapatkan total revenue
order_revenue_df = order_items_df.merge(orders_df, on="order_id", how="left")

# Hitung total penjualan dan revenue per bulan
sales_trend = order_revenue_df.groupby("order_purchase_month").agg({
    "order_id": "count",
    "price": "sum"
}).rename(columns={"order_id": "total_orders", "price": "total_revenue"})

# Tampilkan hasil
sales_trend.tail(5)  # 5 bulan terakhir

**Insight:**
- Tren order dan revenue menunjukkan pola musiman atau tren naik/turun.
- Kita bisa melihat bulan dengan penjualan tertinggi dan terendah.

### Produk Terlaris dan Kurang Laku

In [ ]:
# Hitung jumlah produk terjual
product_sales = order_items_df.groupby("product_id")["order_id"].count().reset_index()
product_sales = product_sales.merge(products_df, on="product_id", how="left")

# Urutkan produk berdasarkan jumlah order
top_selling_products = product_sales.sort_values("order_id", ascending=False).head(5)
least_selling_products = product_sales.sort_values("order_id", ascending=True).head(5)

# Tampilkan hasil
top_selling_products[["product_id", "product_category_name", "order_id"]]

**Insight**
- Produk dengan penjualan tertinggi bisa menjadi fokus promosi.
- Produk dengan penjualan terendah bisa dievaluasi untuk penghapusan atau perbaikan strategi.

###  Analisis Demografi Pelanggan

In [ ]:
# Hitung jumlah pelanggan per state
customer_distribution = customers_df["customer_state"].value_counts()

# Tampilkan hasil
customer_distribution.head(5)  # 5 negara bagian dengan pelanggan terbanyak

**Insight**
- Negara bagian dengan pelanggan terbanyak bisa menjadi fokus pemasaran.
- Kita bisa melihat apakah ada daerah dengan potensi pelanggan baru.

## Visualization & Explanatory Analysis

### Pertanyaan 1: Bagaimana performa penjualan dan revenue perusahaan dalam beberapa bulan terakhir?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Konversi period ke datetime agar bisa diproses di grafik
sales_trend.index = sales_trend.index.to_timestamp()

# Plot tren penjualan dan revenue
fig, ax1 = plt.subplots(figsize=(12, 5))

# Plot total order
ax1.set_title("Tren Penjualan & Revenue Perusahaan", fontsize=14)
ax1.set_xlabel("Bulan", fontsize=12)
ax1.set_ylabel("Total Orders", fontsize=12, color="tab:blue")
ax1.plot(sales_trend.index, sales_trend["total_orders"], marker="o", linestyle="-", color="tab:blue", label="Total Orders")
ax1.tick_params(axis="y", labelcolor="tab:blue")

# Tambahkan sumbu kedua untuk revenue
ax2 = ax1.twinx()
ax2.set_ylabel("Total Revenue (R$)", fontsize=12, color="tab:orange")
ax2.plot(sales_trend.index, sales_trend["total_revenue"], marker="o", linestyle="-", color="tab:orange", label="Total Revenue")
ax2.tick_params(axis="y", labelcolor="tab:orange")

fig.tight_layout()
plt.show()

**Insight:**
- Kita bisa melihat tren penjualan apakah naik atau turun dari bulan ke bulan.
- Revenue juga bisa dibandingkan dengan jumlah order untuk melihat harga rata-rata per order.

### Pertanyaan 2: Produk apa yang paling banyak dan paling sedikit terjual?

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(y=top_selling_products["product_category_name"], 
            x=top_selling_products["order_id"], 
            hue=top_selling_products["product_category_name"],  # Menambahkan hue
            dodge=False,  # Hindari pemisahan warna per kategori
            legend=False)  # Matikan legend karena tidak perlu
plt.xlabel("Jumlah Terjual", fontsize=12)
plt.ylabel("Kategori Produk", fontsize=12)
plt.title("Top 5 Produk Terlaris", fontsize=14)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(y=least_selling_products["product_category_name"], 
            x=least_selling_products["order_id"], 
            hue=least_selling_products["product_category_name"], 
            dodge=False, 
            legend=False)
plt.xlabel("Jumlah Terjual", fontsize=12)
plt.ylabel("Kategori Produk", fontsize=12)
plt.title("Top 5 Produk dengan Penjualan Terendah", fontsize=14)
plt.show()


**Insight:**
- Produk terlaris menunjukkan kategori yang paling diminati pelanggan.
- Produk kurang laku mungkin perlu dievaluasi, baik dari segi harga, pemasaran, atau stok.

### Pertanyaan 3: Bagaimana demografi pelanggan yang kita miliki?

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x=customer_distribution.index, y=customer_distribution.values)
plt.xlabel("Lokasi Pelanggan", fontsize=12)
plt.ylabel("Jumlah Pelanggan", fontsize=12)
plt.title("Distribusi Pelanggan Berdasarkan Lokasi", fontsize=14)
plt.xticks(rotation=45)
plt.show()

### Insight
- Sebagian besar pelanggan berasal dari negara bagian São Paulo (SP).
- Bisa menjadi peluang untuk fokus strategi pemasaran di area dengan pelanggan terbanyak.

## Conclusion

- Berdasarkan analisis data, performa penjualan dan revenue perusahaan menunjukkan pola fluktuatif dalam beberapa bulan terakhir. Terdapat periode tertentu di mana terjadi peningkatan jumlah pesanan dan pendapatan, yang mengindikasikan adanya pola musiman dalam aktivitas penjualan. Dengan memahami pola ini, perusahaan dapat merancang strategi promosi yang lebih efektif pada waktu-waktu tertentu guna memaksimalkan keuntungan. Selain itu, metode pembayaran yang paling sering digunakan oleh pelanggan juga memberikan wawasan bagi perusahaan untuk mengoptimalkan sistem transaksi.

- Dari segi produk, terdapat beberapa kategori produk yang sangat diminati oleh pelanggan, sementara beberapa kategori lainnya memiliki angka penjualan yang lebih rendah. Hal ini menunjukkan adanya preferensi pelanggan terhadap jenis produk tertentu yang bisa menjadi dasar dalam pengelolaan stok dan strategi pemasaran. Dengan mengetahui produk terlaris dan kurang diminati, perusahaan dapat menyesuaikan strategi inventaris serta menjalankan promosi yang lebih tepat sasaran untuk meningkatkan penjualan produk yang kurang laku di pasaran.

- Terakhir, analisis demografi pelanggan mengungkapkan bahwa mayoritas pelanggan berasal dari kota-kota besar, terutama dari negara bagian São Paulo (SP). Persebaran geografis pelanggan yang tidak merata memberikan peluang bagi perusahaan untuk menargetkan daerah tertentu dalam kampanye pemasaran. Dengan memanfaatkan data lokasi ini, perusahaan dapat mengembangkan strategi yang lebih personalisasi, seperti promosi berbasis lokasi dan penyesuaian logistik agar lebih efisien dalam memenuhi permintaan pelanggan. Secara keseluruhan, hasil analisis ini memberikan wawasan berharga yang dapat membantu perusahaan dalam pengambilan keputusan strategis untuk meningkatkan penjualan dan kepuasan pelanggan.

In [23]:
import os
import pandas as pd

# Path dataset
dataset_path = "C:/Users/Aldi/dicoding proyek/ecommerce_dataset/submission/data"

# Load dataset
customers_df = pd.read_csv(os.path.join(dataset_path, "customers_dataset.csv"))
orders_df = pd.read_csv(os.path.join(dataset_path, "orders_dataset.csv"), parse_dates=["order_purchase_timestamp"])
order_items_df = pd.read_csv(os.path.join(dataset_path, "order_items_dataset.csv"))
products_df = pd.read_csv(os.path.join(dataset_path, "products_dataset.csv"))
product_translation_df = pd.read_csv(os.path.join(dataset_path, "product_category_name_translation.csv"))
sellers_df = pd.read_csv(os.path.join(dataset_path, "sellers_dataset.csv"))

# Gabungkan order_items_df dengan produk
merged_df = order_items_df.merge(products_df, on="product_id", how="left")
merged_df = merged_df.merge(product_translation_df, on="product_category_name", how="left")
merged_df = merged_df.merge(orders_df, on="order_id", how="left")
merged_df = merged_df.merge(customers_df, on="customer_id", how="left")
merged_df = merged_df.merge(sellers_df, on="seller_id", how="left")

# Tambahkan kolom order_purchase_month
merged_df["order_purchase_month"] = merged_df["order_purchase_timestamp"].dt.to_period("M")

# Pastikan direktori "submission/dashboard" ada
dashboard_path = "C:/Users/Aldi/dicoding proyek/ecommerce_dataset/submission/dashboard"
os.makedirs(dashboard_path, exist_ok=True)

# Simpan dataset utama
merged_df.to_csv(os.path.join(dashboard_path, "main_data.csv"), index=False)

print("✅ File main_data.csv berhasil dibuat di:", dashboard_path)


✅ File main_data.csv berhasil dibuat di: C:/Users/Aldi/dicoding proyek/ecommerce_dataset/submission/dashboard
